# Pozo Finito

Sigo las cuentas de [Wikipedia](https://en.wikipedia.org/wiki/Finite_potential_well).

La unica diferencia es que tomo el pozo entre $-L$ y $L$ en vez de entre $-\frac{L}{2}$ y $\frac{L}{2}$.

Hay que encontrar las soluciones de 
$$\sqrt{\left(\frac{u_0}{v}\right)^2 - 1} = \begin{cases} \tan{(v)} & \text{(Soluciones Simétricas)} \\
      -\cot{(v)} & \text{(Soluciones Antisimétricas)} \end{cases},$$
donde $u_{0}^{2} = \frac{2 m  L^2 V}{\hbar^2}$.

Si tomamos $$LHS(v) = \sqrt{\left(\frac{u_0}{v}\right)^2 - 1}$$ y $$RHS(v) = \begin{cases} \tan{(v)} & \text{(Soluciones Simetricas)} \\ -\cot{(v)} & \text{(Soluciones Antisimetricas)} \end{cases}$$

podemos buscar los picos de $$f(v) = \frac{1}{|LHS(v) - RHS(v)| + \delta}$$ los cuales corresponden a las soluciones ($\delta \ll 1$).

In [1]:
%matplotlib inline
import numpy as np
from scipy import constants, signal
import matplotlib.pyplot as plt
import ipywidgets as widgets

def dE(V, p, a=1.39, d=0.0, N=10000, delta=1e-5, offset=1e-3, plot=True):
    Ni = np.floor((p + 3)/2).astype(int) - 1 
    Nf = Ni + 1
    
    eV = constants.electron_volt
    agm = constants.angstrom
    hbar = constants.hbar
    m = constants.electron_mass
    
    V_mks = V * eV
    a_mks = a * agm
    
    L = 0.5*a_mks*(p+2+d)
    u0 = (L/hbar)*np.sqrt(2*m*V_mks) 
    v = np.linspace(offset, u0, N)
    
    num_niveles = np.ceil(2*u0/np.pi).astype(int)
    
    lhs = np.sqrt((u0/v)**2 - 1)
    cond_s = np.mod(np.floor(2*v/np.pi).astype(int), 2) == 0
    cond_a = np.mod(np.floor(2*v/np.pi).astype(int), 2) == 1
    rhs = np.piecewise(v, [cond_s, cond_a], [lambda x: np.tan(x), lambda x: np.tan(x - 0.5*np.pi)])
    
    auxf = 1/(np.abs(lhs - rhs)+delta)
    
    roots = signal.find_peaks(auxf)[0]
    
    levels_mks = (v[roots]/L)**2 * hbar**2/(2*m)
    levels_eV = levels_mks/eV
    dE = np.round(levels_eV[Nf] - levels_eV[Ni], 3)
    
    if (plot is True):
        plt.figure(figsize=(10, 6))
        plt.title(r'$\Delta E_{' + str(Ni+1) + r'\rightarrow' + str(Nf+1) + r'} = ' + str(dE) + r'$' + 'eV' )
        plt.plot(v, lhs, color='dimgrey', label=r'$\sqrt{\left(\frac{u_{0}}{v}\right)^{2}-1}$')
        plt.ylim(0, np.sqrt((4*u0/np.pi)**2 - 1))
        plt.plot(v, np.ma.masked_outside(np.tan(v), 0.0, 2*np.sqrt((4*u0/np.pi)**2 - 1)), color='tab:red', label=r'$\tan{(v)}$' + " (Soluciones Simétricas)")
        plt.plot(v, np.ma.masked_outside(np.tan(v - 0.5*np.pi), 0.0, 2*np.sqrt((4*u0/np.pi)**2 - 1)), color='tab:blue', label=r'$- \cot{(v)}$' + " (Soluciones Antisimétricas)")
        plt.xlabel(r'$v$')
        plt.legend(loc='upper right')
        plt.show()      
    return dE

def fit_V(dE5, dE7, dE9, Vmin, Vmax, N=1000):
    model_dE = np.frompyfunc(lambda V, p: dE(V, p, plot=False), 2, 1)
    
    p_a = np.arange(5.0, 11.0, 2.0)
    dE_a = np.array([dE5, dE7, dE9])
    V_a = np.linspace(Vmin, Vmax, N)
    chi_a = (model_dE(V_a, 5) - dE5)**2 + (model_dE(V_a, 7) - dE7)**2 + (model_dE(V_a, 9) - dE9)**2
    arg_min = np.argmin(chi_a)
    chi_opt = np.round(chi_a[arg_min], 4)
    V_opt = np.round(V_a[arg_min], 3)
    
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    ax[0].plot(p_a, model_dE(V_opt, p_a), "ro", label="Ajuste")
    ax[0].plot(p_a, dE_a, "bo", label="Datos Experimentales")
    ax[0].set_xticks([5, 7, 9])
    ax[0].set_xlabel(r'$p$')
    ax[0].set_ylabel(r'$\Delta E$' + '[eV]', rotation=0)
    ax[0].legend(loc='upper right')
    ax[1].plot(V_a, chi_a)
    ax[1].set_title(r'$V_{opt} = $' + str(V_opt) + ' eV  ' + r'$\chi_{opt} = $' + str(chi_opt))
    ax[1].set_xlabel(r'$V$' + '[eV]')
    ax[1].set_ylabel(r'$\chi^{2}$', rotation=0)
    plt.show() 
    return V_opt

interactive_dE = lambda V, p, a, d: dE(V, p, a=a, d=d)
interactive_fit = lambda dE5, dE7, dE9, Vmin, Vmax: fit_V(dE5, dE7, dE9, Vmin, Vmax, N=1000)

widgets.interact(interactive_dE, V=widgets.FloatSlider(min=0.1, max=100.0, step=0.1, value=8.0, description=r'$V[eV]$'),
                 p=widgets.IntSlider(min=3, max=9, step=2, value=5, description=r'$p$'),
                 a=widgets.FloatSlider(min=0.5, max=2.0, step=0.01, value=1.39, description=r'$a [Å]$'),
                 d=widgets.FloatSlider(min=0.0, max=0.5, step=0.1, value=0.0, description=r'$d$'))

#widgets.interact(interactive_fit, dE5=widgets.BoundedFloatText(min=0.1, max=10.0, step=0.001, value=2.50, description=r'$\Delta E_{4 \rightarrow 5} [eV] (p=5)$', readout_format='.3f'),
#                 dE7=widgets.BoundedFloatText(min=0.1, max=10.0, step=0.001, value=2.1, description=r'$\Delta E_{5 \rightarrow 6} [eV] (p=7)$', readout_format='.3f'),
#                 dE9=widgets.BoundedFloatText(min=0.1, max=10.0, step=0.001, value=1.8, description=r'$\Delta E_{6 \rightarrow 7} [eV] (p=9)$', readout_format='.3f'),
#                 Vmin=widgets.BoundedFloatText(min=1.0, max=20.0, step=0.1, value=8.0, description=r'$V_{min} [eV]$'),
#                 Vmax=widgets.BoundedFloatText(min=1.0, max=20.0, step=0.1, value=12.0, description=r'$V_{max} [eV]$'))

interactive(children=(FloatSlider(value=8.0, description='$V[eV]$', min=0.1), IntSlider(value=5, description='…

interactive(children=(BoundedFloatText(value=2.5, description='$\\Delta E_{4 \\rightarrow 5} [eV] (p=5)$', max…

<function __main__.<lambda>(dE5, dE7, dE9, Vmin, Vmax)>